In [1]:
from pyspark.sql import SparkSession

sc = SparkSession.builder.getOrCreate()

23/06/09 12:20:02 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.100.230 instead (on interface wlp5s0)
23/06/09 12:20:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/09 12:20:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [81]:
from numpy import log
from pandas import DataFrame, Series, concat
from numpy.random import default_rng, seed
from numpy import select, full

seed(117)

N = 1000000

features = DataFrame({
  'a': default_rng().integers(10, 100, size=N),
  'dummy1': full(shape=N, fill_value=1),
  'dummy2': full(shape=N, fill_value=1),
  'dummy3': full(shape=N, fill_value=1),
})

label = Series(log(features['a'].values), name='label')
censor = Series(select(
 [
  label > 5 ,
  label <=5
 ],
 [1, 0] 
), name='censor')

data = sc.createDataFrame(concat([features, label, censor], axis=1))
data.show()

/home/aadi/miniconda3/envs/spark_env/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/aadi/miniconda3/envs/spark_env/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


23/06/09 13:06:37 WARN TaskSetManager: Stage 1172 contains a task of very large size (1788 KiB). The maximum recommended task size is 1000 KiB.
+---+------+------+------+------------------+------+
|  a|dummy1|dummy2|dummy3|             label|censor|
+---+------+------+------+------------------+------+
| 20|     1|     1|     1| 2.995732273553991|     0|
| 47|     1|     1|     1|3.8501476017100584|     0|
| 63|     1|     1|     1| 4.143134726391533|     0|
| 44|     1|     1|     1| 3.784189633918261|     0|
| 15|     1|     1|     1|  2.70805020110221|     0|
| 17|     1|     1|     1| 2.833213344056216|     0|
| 75|     1|     1|     1|  4.31748811353631|     0|
| 27|     1|     1|     1| 3.295836866004329|     0|
| 53|     1|     1|     1| 3.970291913552122|     0|
| 38|     1|     1|     1|3.6375861597263857|     0|
| 22|     1|     1|     1| 3.091042453358316|     0|
| 98|     1|     1|     1| 4.584967478670572|     0|
| 81|     1|     1|     1| 4.394449154672439|     0|
| 77|   

In [82]:
from pyspark.ml import feature

va = feature.VectorAssembler(inputCols=['a', 'dummy1', 'dummy2', 'dummy3'], outputCol='features')
data_vec = va.transform(data)
data_vec.head()

23/06/09 13:06:41 WARN TaskSetManager: Stage 1173 contains a task of very large size (1788 KiB). The maximum recommended task size is 1000 KiB.


Row(a=20, dummy1=1, dummy2=1, dummy3=1, label=2.995732273553991, censor=0, features=DenseVector([20.0, 1.0, 1.0, 1.0]))

In [83]:
from pyspark.ml.regression import AFTSurvivalRegression

aft = AFTSurvivalRegression(
  labelCol='label',
  censorCol='censor',
  featuresCol='features'
)

model = aft.fit(data_vec)

23/06/09 13:06:43 WARN TaskSetManager: Stage 1174 contains a task of very large size (1788 KiB). The maximum recommended task size is 1000 KiB.
23/06/09 13:06:43 WARN TaskSetManager: Stage 1176 contains a task of very large size (1788 KiB). The maximum recommended task size is 1000 KiB.
23/06/09 13:06:44 WARN TaskSetManager: Stage 1178 contains a task of very large size (1788 KiB). The maximum recommended task size is 1000 KiB.
23/06/09 13:06:44 WARN TaskSetManager: Stage 1180 contains a task of very large size (1788 KiB). The maximum recommended task size is 1000 KiB.
23/06/09 13:06:44 WARN TaskSetManager: Stage 1182 contains a task of very large size (1788 KiB). The maximum recommended task size is 1000 KiB.
23/06/09 13:06:44 WARN TaskSetManager: Stage 1184 contains a task of very large size (1788 KiB). The maximum recommended task size is 1000 KiB.
23/06/09 13:06:44 WARN TaskSetManager: Stage 1186 contains a task of very large size (1788 KiB). The maximum recommended task size is 10

In [84]:
model.coefficients

DenseVector([0.0113, 0.0, 0.0, 0.0])